<center><a href="https://www.pieriantraining.com/" ><img src="../PTCenteredPurple.png" alt="Pierian Training Logo" /></a></center>


# Помощник с несколькими файлами

Давайте представим, что вас только что взяли в команду руководителей новой компании: ACME Dog Food Corporation. Чтобы помочь ответить на вопросы, давайте создадим помощника с доступом к базе знаний, состоящей из множества корпоративных документов.

In [1]:
from openai import OpenAI

In [3]:
client = OpenAI()

## Uploading Files

In [4]:
import os

In [4]:
os.listdir('02-Multiple-Files/')

['ACME Advertising Policy.pdf',
 'ACME_Dog_Food_Ingredients.pdf',
 'Corporate Travel Policy.pdf']

In [17]:
files_to_upload = ['plan_in_sign.docx',
 'plan_upay.docx',
 'signs.docx']

In [18]:
def upload_assistant_file(filename):
    file = client.files.create(
      file=open('02-Assistant-with-Knowledge-Retrieval/'+filename, "rb"),
      purpose='assistants'
    )
    print(file.id)

In [19]:
for file in files_to_upload:
    upload_assistant_file(file)

file-cUcDly7V6HXXtlqNKpZ8fZgj
file-rT4AxNGvSHIx3aHpXp9TICuk
file-fkfew1ulliZgW2Fq8bVxgWFa


In [20]:
for file in client.files.list():
    print(f"Имя файла: {file.filename}, ID файла: {file.id}")

Имя файла: signs.docx, ID файла: file-fkfew1ulliZgW2Fq8bVxgWFa
Имя файла: plan_upay.docx, ID файла: file-rT4AxNGvSHIx3aHpXp9TICuk
Имя файла: plan_in_sign.docx, ID файла: file-cUcDly7V6HXXtlqNKpZ8fZgj


In [21]:
file_ids = [
    "file-fkfew1ulliZgW2Fq8bVxgWFa",
    "file-rT4AxNGvSHIx3aHpXp9TICuk",
    "file-cUcDly7V6HXXtlqNKpZ8fZgj"
]

vector_store = client.beta.vector_stores.create(
  name="Астрологические интерпритации"
)

In [38]:
vs_id=(vector_store.id)

In [24]:
from datetime import datetime

def load_files_to_vector_store(vector_store_id, file_ids):
    """
    Загружает файлы в хранилище векторов.

    Args:
        vector_store_id: ID хранилища векторов.
        file_ids: Список ID файлов.

    Returns:
        Объект ответа от API 'vector_stores.file_batches.create'.
    """
    batch_name = f"Загрузка файлов - {datetime.now().isoformat()}"
    print(f"Начинаем загрузку файлов в хранилище векторов '{vector_store_id}' под названием '{batch_name}'.")

    batch = client.beta.vector_stores.file_batches.create(
        vector_store_id=vector_store_id,
        file_ids=file_ids
    )

    print(batch_name)
    print(f"Статус загрузки: {batch.status}")
    print(f"Количество файлов в пакете: {batch.file_counts.total}")
    return batch


In [25]:
batch = load_files_to_vector_store(vs_id, file_ids)

Начинаем загрузку файлов в хранилище векторов 'vs_0iQPPjC37SiR3j0SfZpBi0FF' под названием 'Загрузка файлов - 2024-05-11T22:55:35.699846'.
Загрузка файлов - 2024-05-11T22:55:35.699846
Статус загрузки: in_progress
Количество файлов в пакете: 3


In [27]:
def check_file_batch_status(vector_store_id, batch_id):
  """
  Проверяет статус загрузки файлов в хранилище векторов.

  Args:
    vector_store_id: ID хранилища векторов.
    batch_id: ID пакета загрузки файлов.

  Returns:
    Статус загрузки (например, "completed", "in_progress", "failed").
  """

  vector_store_retrieve_file_batch = client.beta.vector_stores.file_batches.retrieve(
      vector_store_id=vector_store_id,
      batch_id=batch_id
  )

  return vector_store_retrieve_file_batch.status

# Пример использования
vector_store_id = "vs_0iQPPjC37SiR3j0SfZpBi0FF"
batch_id = batch.id

status = check_file_batch_status(vs_id, batch_id)

if status == "completed":
  print("Файлы успешно загружены!")
else:
  print(f"Результат: {status}")

Файлы успешно загружены!


## Проверяем что в нашей ВекБД хранятся именно наши файлы

In [50]:
vector_store_file_batch = client.beta.vector_stores.file_batches.retrieve(
  vector_store_id=vs_id,
  batch_id=batch.id
)
print(vector_store_file_batch)

VectorStoreFileBatch(id='vsfb_dd3b90edf3a54a27a68f9cea35fca329', created_at=1715457582, file_counts=FileCounts(cancelled=0, completed=3, failed=0, in_progress=0, total=3), object='vector_store.file_batch', status='completed', vector_store_id='vs_0iQPPjC37SiR3j0SfZpBi0FF')


## Adding Files for the Assistant 

1. **Assistant-Level Attachment**: When you attach a file at this level, it becomes a part of the Assistant's general knowledge base. This means the Assistant can access this information when responding to any query.

## Проверяем список своих агентов

In [70]:
from openai import OpenAI

def print_assistant_info():

    my_assistants = client.beta.assistants.list(
        order="desc",
        limit="20",
    )

    for assistant in my_assistants.data:
        assistant_id = assistant.id
        assistant_instructions = assistant.instructions
        assistant_name = assistant.name

        print(f"Assistant ID: {assistant_id}")
        print(f"Instructions: {assistant_instructions}")
        print(f"Name: {assistant_name}")
        print("---")

# Вызов функции для вывода информации об ассистентах
print_assistant_info()

Assistant ID: asst_F3h4FrIhhkN9gpzfvK3aWxKd
Instructions: Вы отвечаете на вопросы по тексту прелоставленных вам докуменов, отвечайте развернуто.
Name: None
---
Assistant ID: asst_5wao97wKjIYCw2l0g0nS4cxI
Instructions: Вы помощник в программировании языка Python и помогаете настроить VSCode,dds общаетесь с новичками в программировании.
Name: помощник
---
Assistant ID: asst_yOEj5bDErckIQmWM0zgMdSVj
Instructions: Вы отвечаете развернуто по вопросам ведической астрологии, ваш ответ строиться строго на предоставленных вам текстовых документах
Name: Бот ведической астрологии
---


## Удаление ассистентов

In [35]:
assistant_ids = ["asst_OywtDLNTG9xNIr9uzgBqIxdo", "asst_id9UIqBRt68Ucthq2VJ7J80s"]

In [36]:
for assistant_id in assistant_ids:
    response = client.beta.assistants.delete(assistant_id)
    print(response)

AssistantDeleted(id='asst_OywtDLNTG9xNIr9uzgBqIxdo', deleted=True, object='assistant.deleted')
AssistantDeleted(id='asst_id9UIqBRt68Ucthq2VJ7J80s', deleted=True, object='assistant.deleted')


## Создаем ассистента по нашу задачу, или делайем обновление предыдущего

In [48]:
# Add the file to the assistant
assistant = client.beta.assistants.update(
  'asst_F3h4FrIhhkN9gpzfvK3aWxKd',
  description=" Агент RAG по документам астрологии",
  instructions="Вы отвечаете на вопросы по тексту прелоставленных вам докуменов, отвечайте развернуто.",
  model="gpt-3.5-turbo-1106", # CHECK FOR ERROR IF YOU PROVIDE THE WRONG MODEL
  tools=[{"type": "file_search"}],
  tool_resources={"file_search": {"vector_store_ids": [vs_id]}},
  temperature=0.3
)

## Run a Thread

In [51]:
thread = client.beta.threads.create(
    messages=[
    {
      "role": "user",
      "content": "Здравстуйте, вам сейчас будут заданы вопросы об астрологии.",
    },
    {
      "role": "assistant",
      "content": "Привет, я очень рад помочь, пожалуйста я жду ваши вопросы.",
    }
  ],
    tool_resources={"file_search": {"vector_store_ids": [vs_id]}},
)

In [52]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="РАсскажите мне про планету ЛУНУ в Козероге, а так же какие Упайи для Луны требуются?"
)   

In [53]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

In [54]:
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id,
)
print(run.status)

completed


Once the run is completed, let's get the messages

In [55]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id, order="asc",
)

In [56]:
for message in messages:
    print(message)
    print('\n')

Message(id='msg_aS7t6b9euncZ0mFA2e0tj5pD', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Здравстуйте, вам сейчас будут заданы вопросы об астрологии.'), type='text')], created_at=1715504392, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_03uu8MmiyzFpDiskBoIX19jO')


Message(id='msg_yXfbDrEyZYWYo7rQgYPz6c4s', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Привет, я очень рад помочь, пожалуйста я жду ваши вопросы.'), type='text')], created_at=1715504392, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id=None, status=None, thread_id='thread_03uu8MmiyzFpDiskBoIX19jO')


Message(id='msg_mfPtkrt81Hxx0pw6340gduWe', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations

In [68]:
run_steps = client.beta.threads.runs.steps.list(
    thread_id=thread.id,
    run_id=run.id
)

In [69]:
import json
print(json.dumps(run_steps.to_dict(), indent=2))

{
  "data": [
    {
      "id": "step_GpF4QLRo5qYfO3NX0znw0ZYw",
      "assistant_id": "asst_F3h4FrIhhkN9gpzfvK3aWxKd",
      "cancelled_at": null,
      "completed_at": 1715506210,
      "created_at": 1715506204,
      "failed_at": null,
      "last_error": null,
      "object": "thread.run.step",
      "run_id": "run_uibEj7JSt0Md7pmtvy2v6Ya0",
      "status": "completed",
      "step_details": {
        "message_creation": {
          "message_id": "msg_m04ydSPUR0FeLAFEqrAuhmYu"
        },
        "type": "message_creation"
      },
      "thread_id": "thread_03uu8MmiyzFpDiskBoIX19jO",
      "type": "message_creation",
      "usage": {
        "completion_tokens": 661,
        "prompt_tokens": 4976,
        "total_tokens": 5637
      },
      "expires_at": null
    },
    {
      "id": "step_SVL45BWPd7ASQRrL2kAKdQM3",
      "assistant_id": "asst_F3h4FrIhhkN9gpzfvK3aWxKd",
      "cancelled_at": null,
      "completed_at": 1715506204,
      "created_at": 1715506202,
      "failed_at": 

In [57]:
def display_thread_messages(messages):
    # EXPECTS MESSAGES IN ASC ORDER!
    for thread_message in messages.data:
        print(thread_message.content[0].text.value)
        print('\n')

In [58]:
display_thread_messages(messages)

Здравстуйте, вам сейчас будут заданы вопросы об астрологии.


Привет, я очень рад помочь, пожалуйста я жду ваши вопросы.


РАсскажите мне про планету ЛУНУ в Козероге, а так же какие Упайи для Луны требуются?


Планета Луна в знаке Козерога указывает на то, что человек может проявляться как холодный, расчетливый и независимый, но в то же время он может быть заботливым, добрым и организованным. Луна в этом положении также указывает на важность мнения и поддержки матери в сфере социальных достижений, а также на стремление доказать свою профессиональность и получить поддержку от матери в карьере .

Что касается Упайи для Луны, рекомендуется делать упайи по понедельникам, читать мантру "ом намо бхагаватэ васудэвайя" 108 раз с утра, поститься по понедельникам, употреблять масло гхи и теплое молоко на ночь, проявлять почтительное уважение к матери и детям, делать пожертвования в детские дома, сажать молочные деревья, носить серебро, лунный камень жемчуг и цвета: молочный, светлый, белый, сере

## Вытаскивание аннотаций (чанки которые испольщовались для ответа модели). Пок еще API в бета тестировании и они часто отсутствуют

In [59]:
# Retrieve the message object
message = client.beta.threads.messages.retrieve(
  thread_id=thread.id,
  message_id="msg_jic0ARrmIZm44VPlPIqWp1PZ"
)
# Extract the message content
message_content = message.content[0].text
annotations = message_content.annotations
citations = []
# Iterate over the annotations and add footnotes
for index, annotation in enumerate(annotations):
    # Replace the text with a footnote
    message_content.value = message_content.value.replace(annotation.text, f' [{index}]')
    # Gather citations based on annotation attributes
    if (file_citation := getattr(annotation, 'file_citation', None)):
        cited_file = client.files.retrieve(file_citation.file_id)
        citations.append(f'[{index}] {file_citation.quote} from {cited_file.filename}')
    elif (file_path := getattr(annotation, 'file_path', None)):
        cited_file = client.files.retrieve(file_path.file_id)
        citations.append(f'[{index}] Click <here> to download {cited_file.filename}')
        # Note: File download functionality not implemented above for brevity
# Add footnotes to the end of the message before displaying to user
message_content.value += '\n' + '\n'.join(citations)

### Важное замечание о цитировании:  

На момент съемок/создания блокнота аннотации все еще часто остаются пустыми, но в будущем вы сможете получить к ним доступ, как описано в документации:


https://platform.openai.com/docs/assistants/how-it-works/managing-threads-and-messages

For example:

    # Retrieve the message object
    message = client.beta.threads.messages.retrieve(
      thread_id="...",
      message_id="..."
    )

    # Extract the message content
    message_content = message.content[0].text
    annotations = message_content.annotations
    citations = []

    # Iterate over the annotations and add footnotes
    for index, annotation in enumerate(annotations):
        # Replace the text with a footnote
        message_content.value = message_content.value.replace(annotation.text, f' [{index}]')

        # Gather citations based on annotation attributes
        if (file_citation := getattr(annotation, 'file_citation', None)):
            cited_file = client.files.retrieve(file_citation.file_id)
            citations.append(f'[{index}] {file_citation.quote} from {cited_file.filename}')
        elif (file_path := getattr(annotation, 'file_path', None)):
            cited_file = client.files.retrieve(file_path.file_id)
            citations.append(f'[{index}] Click <here> to download {cited_file.filename}')
            # Note: File download functionality not implemented above for brevity

    # Add footnotes to the end of the message before displaying to user
    message_content.value += '\n' + '\n'.join(citations)

# Ask another message

In [63]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="РАсскажи мне про знак Скорпион в общих чертах, а так же про упайи Сатурна"
)

In [64]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

In [66]:
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id,
)
print(run.status)

completed


In [67]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id, order="asc",
)

display_thread_messages(messages)

Здравстуйте, вам сейчас будут заданы вопросы об астрологии.


Привет, я очень рад помочь, пожалуйста я жду ваши вопросы.


РАсскажите мне про планету ЛУНУ в Козероге, а так же какие Упайи для Луны требуются?


Планета Луна в знаке Козерога указывает на то, что человек может проявляться как холодный, расчетливый и независимый, но в то же время он может быть заботливым, добрым и организованным. Луна в этом положении также указывает на важность мнения и поддержки матери в сфере социальных достижений, а также на стремление доказать свою профессиональность и получить поддержку от матери в карьере .

Что касается Упайи для Луны, рекомендуется делать упайи по понедельникам, читать мантру "ом намо бхагаватэ васудэвайя" 108 раз с утра, поститься по понедельникам, употреблять масло гхи и теплое молоко на ночь, проявлять почтительное уважение к матери и детям, делать пожертвования в детские дома, сажать молочные деревья, носить серебро, лунный камень жемчуг и цвета: молочный, светлый, белый, сере

----

# Optional: Delete Assistant and All Files

In [92]:
my_assistants = client.beta.assistants.list(
    order="desc",
    limit="20",
)
response = client.beta.assistants.delete(my_assistants.data[0].id)
print(response)

AssistantDeleted(id='asst_lbz8tHzLjrR84E1NvZOWB4Az', deleted=True, object='assistant.deleted')


In [93]:
for file in client.files.list():
    client.files.delete(file.id)

In [94]:
client.files.list()

SyncPage[FileObject](data=[], object='list', has_more=False)